In [4]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from glob import glob

from utils.plotter import Plotter
from utils.data_handler import DataHandler

labels = ["enkf"]
# labels = ["bias20_no_loc_nosat", "bias20_loc_nosat"]
root = lambda x, y: f"./output_{x}/work{y}/"
n_works = len(glob(root(labels[0], "*")))
template = lambda x, y: f"{root(x, y)}/output/"
data_folder = lambda x: template(labels[0], x)
noise_file = lambda x: f"{root(labels[0], x)}/airpressure_noise.nc"
folder = data_folder(0)
output_folder = "figs"

spin_up_time = 7  # In days
start_date = "2014-06-01"
end_date = "2014-08-31"
assimilation_end_date = "2014-06-09"
noos_folder = "/home/mverlaan/einf220/fromDavid/gtsm_openDA_david/stochObserver/noos_cmems_bk"
noos_file = lambda station: os.path.join(noos_folder, f"timeseries_{start_date}_{end_date}_{station}_averaged.noos")
locs_file = "/home/mverlaan/einf220/fromDavid/gtsm_openDA_david/stochModel/input_dflowfm/grid_locs_bk.xyn"
xs, ys, stations = DataHandler.read_xyn(locs_file)

his_file = "full_gtsm_fine_0000_his.nc"
av_file = "full_gtsm_fine_averaged_0000_his.nc"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
for i in range(n_works):
    nc = xr.load_dataset(noise_file(i))
    nc.sel(time=slice(start_date, assimilation_end_date)).interp(y=52.9, x=4.7).p.plot(
        label=f"{i}"
    )

In [5]:
for station in stations[5:11]:
    ts = []
    ws = []
    works = list(range(n_works))
    works = [str(x) for x in works]
    title_func = lambda s: f"Averaged waterlevel at {s}"

    t_final = 0
    loc_name = ""
    for label in labels:
        # for work in works:
        for work in [0]:
            file = os.path.join(template(label, work), av_file)
            nc_his = xr.load_dataset(file)

            stations_his = nc_his.station_name.values
            stations_his = np.array([s.decode("utf-8").strip() for s in stations_his])
            nc_his["station_name"] = ("stations", stations_his)
            station_data = nc_his.where(nc_his.station_name == station).dropna(
                "stations"
            )

            ts.append(station_data.time.squeeze())
            ws.append(station_data.waterlevel.values.squeeze())
            loc_name = station
            t_final = len(station_data.time.values)

    # Observations
    t_slice = slice(None, t_final - spin_up_time)
    obs_ts, obs = DataHandler.read_noos(noos_file(station))
    observations = [obs_ts[t_slice], obs[t_slice]]
    Plotter.plot_series(
        ts,
        ws,
        loc_name,
        title_func,
        obs_data=observations,
        # legends=labels
    )
    plt.show()

# Plotter.__clear__()
# data1 = ws[0][spin_up_time:]
# data2 = obs[t_slice]
# plt.plot(data1)
# plt.plot(data2)
# print(np.std(data1 - data2))

FileNotFoundError: [Errno 2] No such file or directory: '/home/mverlaan/einf220/fromDavid/gtsm_openDA_david/stochObserver/noos_cmems/timeseries_2014-06-01_2014-08-31_lon-99.5_lat-60.0_averaged.noos'

In [10]:
np.sqrt(((data1 - data2) ** 2).mean())

0.7898351476554145

In [ ]:
nc = xr.open_dataset(
    "/scratch-shared/mverlaan/experiment_gtsm/output_enkf/work0/output/full_gtsm_fine_0000_his.nc"
)
nc

In [ ]:
nc_av = xr.open_dataset(
    "/scratch-shared/mverlaan/experiment_gtsm/output_enkf/work0/output/full_gtsm_fine_averaged_0000_his.nc"
)
nc_av